In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1209009/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2_sup")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [5]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_2_6_3,0.999993,0.992985,0.999961,0.999983,0.999987,1.602137,1564.815663,1.140069,8.871115,5.005592,0.000849,1.265755,0.999943,1.319641,41.057324,66.653716,"Hidden Size=[5], regularizer=0.2, learning_rat..."
1,model_2_6_4,0.999993,0.992985,0.999962,0.999982,0.999987,1.635979,1564.748980,1.128131,9.094963,5.111547,0.000848,1.279054,0.999941,1.333506,41.015517,66.611909,"Hidden Size=[5], regularizer=0.2, learning_rat..."
2,model_2_6_6,0.999993,0.992985,0.999962,0.999982,0.999987,1.638269,1564.854515,1.121134,9.117050,5.119092,0.000785,1.279949,0.999941,1.334439,41.012720,66.609112,"Hidden Size=[5], regularizer=0.2, learning_rat..."
3,model_2_6_5,0.999993,0.992985,0.999962,0.999982,0.999987,1.645534,1564.725389,1.129971,9.152860,5.141416,0.000814,1.282784,0.999941,1.337394,41.003870,66.600263,"Hidden Size=[5], regularizer=0.2, learning_rat..."
4,model_2_9_22,0.999992,0.992982,1.000000,0.999970,0.999996,1.742589,1565.336388,0.000103,0.617299,0.308701,0.000818,1.320072,0.999938,1.376270,40.889256,66.485648,"Hidden Size=[5], regularizer=0.2, learning_rat..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3853,model_31_4_1,0.991416,0.990741,0.999932,0.999969,0.999949,1914.824971,2065.341634,18.259495,6.834214,12.546522,0.097767,43.758713,1.008241,45.621612,82.885237,142.610152,"Hidden Size=[12], regularizer=0.05, learning_r..."
3863,model_31_4_0,0.991243,0.990623,0.999930,0.999970,0.999948,1953.277763,2091.533154,18.613952,6.628130,12.621160,0.098306,44.195902,1.008407,46.077413,82.845472,142.570387,"Hidden Size=[12], regularizer=0.05, learning_r..."
3875,model_31_3_24,0.991067,0.990502,0.999532,0.999879,0.999800,1992.587972,2118.555133,29.890566,24.936764,27.413477,0.098868,44.638414,1.008576,46.538763,82.805621,142.530536,"Hidden Size=[12], regularizer=0.05, learning_r..."
3886,model_31_3_23,0.990889,0.990361,0.999577,0.999876,0.999808,2032.178464,2150.140301,27.010738,25.586267,26.298502,0.099572,45.079690,1.008746,46.998826,82.766273,142.491188,"Hidden Size=[12], regularizer=0.05, learning_r..."
